In [1]:
import pandas as pd

In [2]:
#feature engineering
df = pd.read_csv("../preprocessing/train_data.csv")
#time-based features
# Define mapping of month names to numbers
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

df['Month'] = df['Month'].map(month_mapping)

# Sort by Year and Month (ascending order)
df = df.sort_values(by=['Year', 'Month'], ascending=[True, True]).reset_index(drop=True)

# convert month to integer type
df['Month'] = df['Month'].astype(int)

# Create a 'Quarter' feature
df['Quarter'] = ((df['Month'] - 1) // 3) + 1

df['Years_Since_2000'] = df['Year'] - 2000

df['Departures_Growth'] = df.groupby('Country')['Departures'].pct_change()
df['Arrivals_Growth'] = df.groupby('Country')['Arrivals'].pct_change()

# Standardize country names (remove extra spaces)
df['Country'] = df['Country'].str.strip()



In [3]:
# Fill NaN values in the first few rows (likely due to pct_change()) with 0
df['Departures_Growth'] = df.groupby('Country')['Departures_Growth'].fillna(0)
df['Arrivals_Growth'] = df.groupby('Country')['Arrivals_Growth'].fillna(0)


C:\Users\Vianiece\AppData\Local\Temp\ipykernel_33948\3205859310.py:2: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Departures_Growth'] = df.groupby('Country')['Departures_Growth'].fillna(0)
C:\Users\Vianiece\AppData\Local\Temp\ipykernel_33948\3205859310.py:3: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Arrivals_Growth'] = df.groupby('Country')['Arrivals_Growth'].fillna(0)


In [4]:
# Save the updated dataset back to the train CSV
df.to_csv("../preprocessing/train_data.csv", index=False)

print("Updated train_data.csv saved successfully!")


Updated train_data.csv saved successfully!


In [ ]:
import pandas as pd
import calendar  # Importing calendar module

# Function to get number of days in a month
def days_in_month(year, month):
    return calendar.monthrange(year, month)[1]

# Load datasets
df_main = pd.read_csv("../preprocessing/train_data.csv")
df_holidays = pd.read_excel("../Dataset/singapore_holidays_00_25_month.xlsx")

# Merge datasets on Year and Month
df_merged = df_main.merge(df_holidays, on=['Year', 'Month'], how='left')

# Calculate days in each month dynamically
df_merged['Days in Month'] = df_merged.apply(lambda row: days_in_month(row['Year'], row['Month']), axis=1)

# Calculate Holiday Ratio
df_merged['Holiday Ratio'] = df_merged['Total Holidays'] / df_merged['Days in Month']

# Drop unnecessary columns
df_merged.drop(columns=['Days in Month'], inplace=True)

# Save updated dataset
df_merged.to_csv("updated_dataset.csv", index=False)

print(df_merged.head())


   Month  Year         Country  Departures  Arrivals  Quarter  \
0      1  2000   North America     29012.0   26225.0        1   
1      1  2000  United Kingdom     46677.0   45630.0        1   
2      1  2000         Germany     23172.0   24251.0        1   
3      1  2000          France     11098.0   11469.0        1   
4      1  2000          Europe    118255.0  119735.0        1   

   Years_Since_2000  Departures_Growth  Arrivals_Growth  Total Holidays  \
0                 0                0.0              0.0               2   
1                 0                0.0              0.0               2   
2                 0                0.0              0.0               2   
3                 0                0.0              0.0               2   
4                 0                0.0              0.0               2   

   Holiday Ratio  
0       0.064516  
1       0.064516  
2       0.064516  
3       0.064516  
4       0.064516  


In [ ]:
#include inflation data
import pandas as pd

# Load the main dataset
main_df = pd.read_csv("updated_dataset.csv")

# Load the inflation dataset
inflation_df = pd.read_csv("global_inflation_data.csv")

# Transform inflation data from wide to long format
inflation_df = inflation_df.melt(id_vars=["country_name", "indicator_name"], 
                                 var_name="Year", value_name="Inflation")

# Convert "Year" to integer and filter only relevant years (excluding 2002-2004)
inflation_df["Year"] = inflation_df["Year"].astype(int)
inflation_df = inflation_df[
    (inflation_df["Year"].between(2000, 2025)) & 
    (~inflation_df["Year"].isin([2002, 2003, 2004, 2020, 2021, 2022]))
]

# Drop the "indicator_name" column
inflation_df.drop(columns=["indicator_name"], inplace=True)

# Rename 'country_name' to 'Country' for merging consistency
inflation_df.rename(columns={"country_name": "Country"}, inplace=True)

# **Merge inflation data directly using Country and Year**
merged_df = main_df.merge(inflation_df, on=["Country", "Year"], how="left")

# Forward fill missing values in Inflation column
merged_df["Inflation"].fillna(method="ffill", inplace=True)

# Drop rows where Inflation is still missing after forward fill
merged_df = merged_df.dropna(subset=["Inflation"])

# Save the updated dataset
merged_df.to_csv("final_dataset_with_inflation_by_country.csv", index=False)
print("Updated dataset saved as 'final_dataset_with_inflation_by_country.csv'.")


Updated dataset saved as 'final_dataset_with_inflation_by_country.csv'.


C:\Users\Vianiece\AppData\Local\Temp\ipykernel_33948\2877213688.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df["Inflation"].fillna(method="ffill", inplace=True)
C:\Users\Vianiece\AppData\Local\Temp\ipykernel_33948\2877213688.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["Inflation"].fillna(method="ffill", inplace=True)
